In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

### 프로젝트 세팅

In [2]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = '머신러닝/channel_info_classification.dat'

# 교차검증 횟수
cv_count = 2

# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

# 평가 결과를 담을 리스트
# 필요하다면 다른 것도 만들기
f1_score_list = []

# 학습 모델 이름
model_name_list = []

### 데이터 준비
- 데이터를 불러온 뒤 필요한 전처리를 모두 완료하고 입력 데이터는 **train_X**, 정답(결과) 데이터는 **train_y**라는 변수에 담아 학습에 사용할 수 있도록 준비

In [3]:
# 데이터를 읽어온다.
train_df = pd.read_csv('train_cleaned6.csv')
test_df = pd.read_csv('test_cleaned6.csv')

display(train_df)
display(test_df)

,기준년월,ID,인입일수_ARS_R6M,인입월수_ARS_R6M,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,방문후경과월_PC_R6M,방문후경과월_앱_R6M,방문횟수_PC_B0M,방문일수_PC_B0M,...,방문일수_PC_R6M_30,방문횟수_앱_R6M_1,방문횟수_앱_R6M_10,방문횟수_앱_R6M_20,방문횟수_앱_R6M_30,방문횟수_앱_R6M_40,방문횟수_앱_R6M_50,방문횟수_앱_R6M_60,방문횟수_앱_R6M_70,방문횟수_앱_R6M_80
0,201807,TRAIN_000000,8,6,2,6,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
1,201807,TRAIN_000001,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
2,201807,TRAIN_000002,1,1,2,5,0,0,5,4,...,False,False,False,False,True,False,False,False,False,False
3,201807,TRAIN_000003,10,6,2,6,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
4,201807,TRAIN_000004,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
2399996,201812,TRAIN_399996,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
2399997,201812,TRAIN_399997,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
2399998,201812,TRAIN_399998,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False


,기준년월,ID,인입일수_ARS_R6M,인입월수_ARS_R6M,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,방문후경과월_PC_R6M,방문후경과월_앱_R6M,방문횟수_PC_B0M,방문일수_PC_B0M,...,방문일수_PC_R6M_30,방문횟수_앱_R6M_1,방문횟수_앱_R6M_10,방문횟수_앱_R6M_20,방문횟수_앱_R6M_30,방문횟수_앱_R6M_40,방문횟수_앱_R6M_50,방문횟수_앱_R6M_60,방문횟수_앱_R6M_70,방문횟수_앱_R6M_80
0,201807,TEST_00000,0,0,0,0,1,6,0,0,...,False,True,False,False,False,False,False,False,False,False
1,201807,TEST_00001,1,1,0,0,6,0,0,0,...,False,True,False,False,False,False,False,False,False,False
2,201807,TEST_00002,1,1,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
3,201807,TEST_00003,2,1,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
4,201807,TEST_00004,1,1,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
599996,201812,TEST_99996,0,0,0,0,6,0,0,0,...,False,False,True,False,False,False,False,False,False,False
599997,201812,TEST_99997,0,0,0,0,6,6,0,0,...,False,True,False,False,False,False,False,False,False,False
599998,201812,TEST_99998,0,0,0,0,6,0,0,0,...,False,False,False,False,False,True,False,False,False,False


In [4]:
train_df = train_df.drop_duplicates("ID")

In [6]:
# # 두 데이터 프레임을 합친다.
# df1 = pd.concat([train_df, test_df])
# df1.reset_index(inplace=True, drop=True)

In [5]:
df1 = train_df

In [6]:
# ID 컬럼 제거
df1 = df1.drop(columns=['ID'])

In [7]:
# 입력 변수(X)와 타깃 변수(y)로 데이터를 분리한다.
X = df1.drop('Segment', axis=1)
y = df1['Segment']

# 문자열 -> 숫자
encoder1 = LabelEncoder()
encoder1.fit(y)
y2 = encoder1.transform(y)

# 입력 데이터 표준화
scaler1 = StandardScaler()
scaler1.fit(X)
X2 = scaler1.transform(X)

# 학습용과 검증용으로 나눈다.
train_X = X2
train_y = y2

### 기본 모델 사용하기
- 기본 모델 중에 만족하는 것을 찾았다면 하이퍼 파라미터 튜닝 과정은 생략하기

In [ ]:
knn_basic_model = KNeighborsClassifier()

In [ ]:
knn_basic_model.fit(train_X, train_y)

KNeighborsClassifier()

In [ ]:
# knn_basic_model.score(train_X, train_y)

In [ ]:
# %%time

# # KNN
# knn_basic_model = KNeighborsClassifier()

# # 교차 검증을 수행한다.
# r1 = cross_val_score(knn_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# # 평가 결과를 담아준다.
# f1_score_list.append(r1.mean())

# # 학습 모델 이름을 담아준다.
# model_name_list.append("KNN Basic")

In [ ]:
%%time

# LogisticRegression
lr_basic_model = LogisticRegression(multi_class='multinomial')

# 교차 검증을 수행한다.
r1 = cross_val_score(lr_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())

# 학습 모델 이름을 담아준다.
model_name_list.append("LogisticRegression Basic")

CPU times: user 39.7 s, sys: 173 ms, total: 39.9 s
Wall time: 29.8 s


In [8]:
lr_basic_model = LogisticRegression(multi_class='multinomial')

In [9]:
lr_basic_model.fit(train_X, train_y)

LogisticRegression(multi_class='multinomial')

In [10]:
test_df = pd.read_csv('test_cleaned.csv')

In [11]:
test_df = test_df.drop(columns=['ID'])
test_df = test_df.drop('Segment', axis=1)
test_df = scaler1.transform(test_df)

In [12]:
pred_y = lr_basic_model.predict(test_df)

In [13]:
unique_values, counts = np.unique(pred_y, return_counts=True)

In [27]:
dict(zip(unique_values, counts))

{'C': 890, 'D': 9095, 'E': 590015}

In [14]:
pred_y = encoder1.inverse_transform(pred_y)

In [15]:
unique_values, counts = np.unique(pred_y, return_counts=True)

In [16]:
print(dict(zip(unique_values, counts)))

{'C': 890, 'D': 9095, 'E': 590015}


In [17]:
len(test_df), len(pred_y)


(600000, 600000)

In [18]:
test_df = pd.read_csv('test_cleaned.csv')

In [19]:
test_df['Segment'] = pred_y

In [20]:
test_df = test_df.drop_duplicates("ID")

In [22]:
submit = pd.read_csv("open/sample_submission.csv")

In [23]:
submit['Segment'] = test_df['Segment']

In [24]:
submit.to_csv("submit.csv", index=False)

In [25]:
submit = pd.read_csv("submit.csv")

In [26]:
submit

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,E


In [ ]:
# %%time

# # SVC
# svc_basic_model = SVC(probability=True)

# # 교차 검증을 수행한다.
# r1 = cross_val_score(svc_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# # 평가 결과를 담아준다.
# f1_score_list.append(r1.mean())

# # 학습 모델 이름을 담아준다.
# model_name_list.append("SVC Basic")

In [ ]:
%%time

# DecisionTree
tree_basic_model = DecisionTreeClassifier()

# 교차 검증을 수행한다.
r1 = cross_val_score(tree_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())

# 학습 모델 이름을 담아준다.
model_name_list.append("DecisionTree Basic")

CPU times: user 2.33 s, sys: 21 ms, total: 2.35 s
Wall time: 3.82 s


In [ ]:
%%time

# RandomForest
rf_basic_model = RandomForestClassifier()

# 교차 검증을 수행한다.
r1 = cross_val_score(rf_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())

# 학습 모델 이름을 담아준다.
model_name_list.append("RandomForest Basic")

CPU times: user 43.4 s, sys: 653 ms, total: 44 s
Wall time: 49.5 s


In [ ]:
%%time

# AdaBoost
ada_basic_model = AdaBoostClassifier()

# 교차 검증을 수행한다.
r1 = cross_val_score(ada_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())

# 학습 모델 이름을 담아준다.
model_name_list.append("AdaBoost Basic")

CPU times: user 11.4 s, sys: 44.7 ms, total: 11.5 s
Wall time: 11.5 s


In [ ]:
# %%time

# # GradientBoost
# gb_basic_model = GradientBoostingClassifier()

# # 교차 검증을 수행한다.
# r1 = cross_val_score(gb_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# # 평가 결과를 담아준다.
# f1_score_list.append(r1.mean())

# # 학습 모델 이름을 담아준다.
# model_name_list.append("GradientBoost Basic")

In [ ]:
%%time

# LGBM
lgbm_basic_model = LGBMClassifier(verbose=-1)

# 교차 검증을 수행한다.
r1 = cross_val_score(lgbm_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())

# 학습 모델 이름을 담아준다.
model_name_list.append("LGBM Basic")

CPU times: user 27.4 s, sys: 103 ms, total: 27.5 s
Wall time: 28 s


In [ ]:
%%time

# XGBoost
xgboost_basic_model = XGBClassifier(verbose=-1, silent=True, tree_method='gpu_hist')

# 교차 검증을 수행한다.
r1 = cross_val_score(xgboost_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())

# 학습 모델 이름을 담아준다.
model_name_list.append("XGBoost Basic")

CPU times: user 3.77 s, sys: 197 ms, total: 3.97 s
Wall time: 3.48 s


In [ ]:
f1_score_list

[np.float64(nan),
 np.float64(nan),
 np.float64(nan),
 np.float64(nan),
 np.float64(nan),
 np.float64(nan)]

In [ ]:
%%time

# HardVoting 구성
temp_model1 = KNeighborsClassifier()          # K-최근접 이웃
temp_model2 = LogisticRegression()            # 로지스틱 회귀
temp_model3 = SVC(probability=True)           # 서포트 벡터 머신 (확률 추정 활성화)
temp_model4 = DecisionTreeClassifier()        # 결정 트리
temp_model5 = RandomForestClassifier()        # 랜덤 포레스트
temp_model6 = AdaBoostClassifier()            # AdaBoost
temp_model7 = GradientBoostingClassifier()    # 그래디언트 부스팅
temp_model8 = LGBMClassifier(verbose=-1)      # LightGBM (출력 생략)
temp_model9 = XGBClassifier(verbose=-1, silent=True)  # XGBoost (출력 생략)

# 개별 모델들을 튜플로 묶어 하드보팅 모델 리스트를 만든다.
hard_voting_model_list = [
    ('model1', temp_model1),
    ('model2', temp_model2),
    ('model3', temp_model3),
    ('model4', temp_model4),
    ('model5', temp_model5),
    ('model6', temp_model6),
    ('model7', temp_model7),
    ('model8', temp_model8),
    ('model9', temp_model9),
]

# 하드보팅 방식의 앙상블 모델을 생성한다.
hard_voting_basic_model = VotingClassifier(estimators=hard_voting_model_list, voting='hard')

# 교차 검증을 통해 모델의 F1 점수를 평가한다.
r1 = cross_val_score(hard_voting_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평균 F1 점수를 결과 리스트에 저장한다.
f1_score_list.append(r1.mean())

# 모델 이름을 기록한다.
model_name_list.append("HardVoting Basic")

In [ ]:
# SoftVoting 구성
temp_model1 = KNeighborsClassifier()          # K-최근접 이웃
temp_model2 = LogisticRegression()            # 로지스틱 회귀
temp_model3 = SVC(probability=True)           # 서포트 벡터 머신 (확률 추정 활성화)
temp_model4 = DecisionTreeClassifier()        # 결정 트리
temp_model5 = RandomForestClassifier()        # 랜덤 포레스트
temp_model6 = AdaBoostClassifier()            # AdaBoost
temp_model7 = GradientBoostingClassifier()    # 그래디언트 부스팅
temp_model8 = LGBMClassifier(verbose=-1)      # LightGBM (출력 생략)
temp_model9 = XGBClassifier(verbose=-1, silent=True)  # XGBoost (출력 생략)

# 개별 모델들을 튜플로 묶어 소프트보팅 모델 리스트를 생성한다.
soft_voting_model_list = [
    ('model1', temp_model1),
    ('model2', temp_model2),
    ('model3', temp_model3),
    ('model4', temp_model4),
    ('model5', temp_model5),
    ('model6', temp_model6),
    ('model7', temp_model7),
    ('model8', temp_model8),
    ('model9', temp_model9),
]

# 소프트보팅 방식의 앙상블 모델을 생성한다.
soft_voting_basic_model = VotingClassifier(estimators=soft_voting_model_list, voting='soft')

# 교차 검증을 통해 모델의 F1 점수를 평가한다.
r1 = cross_val_score(soft_voting_basic_model, train_X, train_y, scoring='f1', cv=kfold)

# 평균 F1 점수를 결과 리스트에 저장한다.
f1_score_list.append(r1.mean())

# 모델 이름을 기록한다.
model_name_list.append("SoftVoting Basic")

### 하이퍼파라미터 튜닝

In [ ]:
# KNN
params = {
    'n_neighbors' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

temp_model = KNeighborsClassifier()
knn_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
knn_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(knn_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("KNN Tuning")

In [ ]:
# LogisticRegression
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 0, 10, 100, 1000, 10000]
}

temp_model = LogisticRegression()
lr_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
lr_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(lr_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("LogisticRegression Tuning")

In [ ]:
# SVC
params = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 0, 10, 100, 1000, 10000]
}

temp_model = SVC(probability=True)
svc_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
svc_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(svc_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("SVC Tuning")

In [ ]:
# DecisionTree
params = {
    'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

temp_model = DecisionTreeClassifier()
tree_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
tree_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(tree_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("DecisionTree Tuning")

In [ ]:
# RandomForest
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300]
}

temp_model = RandomForestClassifier()
rf_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
rf_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(rf_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("RandomForest Tuning")

In [ ]:
# AdaBoost
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
}

temp_model = AdaBoostClassifier()
ada_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
ada_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(ada_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("AdaBoost Tuning")

In [ ]:
# GradientBoosting
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
}

temp_model = GradientBoostingClassifier()
gb_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
gb_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(gb_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("GradientBoosting Tuning")

In [ ]:
# LGBM
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
}

temp_model = LGBMClassifier(verbose=-1)
lgbm_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
lgbm_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(lgbm_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("LGBM Tuning")

In [ ]:
# XGBoost
params = {
    'booster' : ['gbtree', 'gblinear'],
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
}

temp_model = XGBClassifier(verbose=-1, silent=True)
xgboost_grid_clf = GridSearchCV(temp_model, param_grid=params, scoring='f1', cv=kfold)
xgboost_grid_clf.fit(train_X, train_y)

# 평가 결과를 담아준다.
f1_score_list.append(xgboost_grid_clf.best_score_)

# 학습 모델 이름을 담아준다.
model_name_list.append("XGBoost Tuning")

In [ ]:
# HardVoting 구성
temp_model1 = KNeighborsClassifier()          # K-최근접 이웃
temp_model2 = LogisticRegression()            # 로지스틱 회귀
temp_model3 = SVC(probability=True)           # 서포트 벡터 머신 (확률 추정 활성화)
temp_model4 = DecisionTreeClassifier()        # 결정 트리
temp_model5 = RandomForestClassifier()        # 랜덤 포레스트
temp_model6 = AdaBoostClassifier()            # AdaBoost
temp_model7 = GradientBoostingClassifier()    # 그래디언트 부스팅
temp_model8 = LGBMClassifier(verbose=-1)      # LightGBM (출력 생략)
temp_model9 = XGBClassifier(verbose=-1, silent=True)  # XGBoost (출력 생략)

# 각 모델에 대해 그리드 서치로 찾은 최적 하이퍼파라미터를 적용한다.
temp_model1.set_params(**knn_grid_clf.best_params_)
temp_model2.set_params(**lr_grid_clf.best_params_)
temp_model3.set_params(**svc_grid_clf.best_params_)
temp_model4.set_params(**tree_grid_clf.best_params_)
temp_model5.set_params(**rf_grid_clf.best_params_)
temp_model6.set_params(**ada_grid_clf.best_params_)
temp_model7.set_params(**gb_grid_clf.best_params_)
temp_model8.set_params(**lgbm_grid_clf.best_params_)
temp_model9.set_params(**xgboost_grid_clf.best_params_)

# 최적화된 개별 모델들을 모아 하드보팅 모델 리스트를 구성한다.
hard_voting_model_list = [
    ('model1', temp_model1),
    ('model2', temp_model2),
    ('model3', temp_model3),
    ('model4', temp_model4),
    ('model5', temp_model5),
    ('model6', temp_model6),
    ('model7', temp_model7),
    ('model8', temp_model8),
    ('model9', temp_model9),
]

# 최적 하이퍼파라미터 기반 하드보팅 모델을 생성한다.
hard_voting_tuning_model = VotingClassifier(estimators=hard_voting_model_list, voting='hard')

# 교차 검증을 통해 모델의 F1 점수를 평가한다.
r1 = cross_val_score(hard_voting_tuning_model, train_X, train_y, scoring='f1', cv=kfold)

# 평균 F1 점수를 결과 리스트에 저장한다.
f1_score_list.append(r1.mean())

# 모델 이름을 기록한다.
model_name_list.append("HardVoting Tuning")

In [ ]:
# SoftVoting 구성
temp_model1 = KNeighborsClassifier()          # K-최근접 이웃
temp_model2 = LogisticRegression()            # 로지스틱 회귀
temp_model3 = SVC(probability=True)           # 서포트 벡터 머신 (확률 추정 활성화)
temp_model4 = DecisionTreeClassifier()        # 결정 트리
temp_model5 = RandomForestClassifier()        # 랜덤 포레스트
temp_model6 = AdaBoostClassifier()            # AdaBoost
temp_model7 = GradientBoostingClassifier()    # 그래디언트 부스팅
temp_model8 = LGBMClassifier(verbose=-1)      # LightGBM (출력 생략)
temp_model9 = XGBClassifier(verbose=-1, silent=True)  # XGBoost (출력 생략)

# 각 모델에 대해 그리드 서치를 통해 찾은 최적 하이퍼파라미터를 적용한다.
temp_model1.set_params(**knn_grid_clf.best_params_)
temp_model2.set_params(**lr_grid_clf.best_params_)
temp_model3.set_params(**svc_grid_clf.best_params_)
temp_model4.set_params(**tree_grid_clf.best_params_)
temp_model5.set_params(**rf_grid_clf.best_params_)
temp_model6.set_params(**ada_grid_clf.best_params_)
temp_model7.set_params(**gb_grid_clf.best_params_)
temp_model8.set_params(**lgbm_grid_clf.best_params_)
temp_model9.set_params(**xgboost_grid_clf.best_params_)

# 최적화된 개별 모델들을 모아 소프트보팅 모델 리스트를 구성한다.
soft_voting_model_list = [
    ('model1', temp_model1),
    ('model2', temp_model2),
    ('model3', temp_model3),
    ('model4', temp_model4),
    ('model5', temp_model5),
    ('model6', temp_model6),
    ('model7', temp_model7),
    ('model8', temp_model8),
    ('model9', temp_model9),
]

# 최적 하이퍼파라미터 기반 소프트보팅 모델을 생성한다.
soft_voting_tuning_model = VotingClassifier(estimators=soft_voting_model_list, voting='soft')

# 교차 검증을 통해 모델의 F1 점수를 평가한다.
r1 = cross_val_score(soft_voting_tuning_model, train_X, train_y, scoring='f1', cv=kfold)

# 평균 F1 점수를 결과 리스트에 저장한다.
f1_score_list.append(r1.mean())

# 모델 이름을 기록한다.
model_name_list.append("SoftVoting Tuning")

In [ ]:
# 모델별 F1 점수 리스트를 딕셔너리 형태로 변환한다.
d1 = {
    'f1 score': f1_score_list
}

# 딕셔너리를 기반으로 DataFrame을 생성하며, 인덱스에는 모델 이름을 지정한다.
result_df = pd.DataFrame(d1, index=model_name_list)

# F1 점수를 기준으로 내림차순 정렬하여 성능이 좋은 모델이 위로 오도록 한다.
result_df.sort_values(by='f1 score', ascending=False, inplace=True)

# 정렬된 결과를 출력한다.
result_df

In [ ]:
# 학습 데이터에 대해 로지스틱 회귀 모델이 예측한 클래스별 확률을 가져온다.
proba_a1 = lr_grid_clf.best_estimator_.predict_proba(train_X)

# 각 샘플이 클래스 0일 확률
a10 = proba_a1[:, 0]

# 각 샘플이 클래스 1일 확률
a20 = proba_a1[:, 1]

# 샘플별로 클래스별 확률을 시각화한다.
plt.scatter(list(range(len(a10))), a10, label='0일 확률')
plt.scatter(list(range(len(a20))), a20, label='1일 확률')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.show()

In [ ]:
# 학습 데이터에 대해 XGBoost 모델이 예측한 클래스별 확률을 가져온다.
proba_a1 = xgboost_grid_clf.best_estimator_.predict_proba(train_X)
# 각 샘플이 클래스 0일 확률
a10 = proba_a1[:, 0]

# 각 샘플이 클래스 1일 확률
a20 = proba_a1[:, 1]

# 샘플별로 클래스별 확률을 시각화한다.
plt.scatter(list(range(len(a10))), a10, label='0일 확률')
plt.scatter(list(range(len(a20))), a20, label='1일 확률')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.show()

In [ ]:
# 학습 데이터에 대해 AdaBoost 모델이 예측한 클래스별 확률을 가져온다.
proba_a1 = ada_grid_clf.best_estimator_.predict_proba(train_X)

# 각 샘플이 클래스 0일 확률
a10 = proba_a1[:, 0]

# 각 샘플이 클래스 1일 확률
a20 = proba_a1[:, 1]

# 샘플별로 클래스별 확률을 시각화한다.
plt.scatter(list(range(len(a10))), a10, label='0일 확률')
plt.scatter(list(range(len(a20))), a20, label='1일 확률')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.show()

In [ ]:
# 일부러 오류가 나는 코드를 넣어준다.
10 / 0

In [ ]:
# 최종 모델로 사용할 XGBoost 분류기를 생성한다.
best_model = XGBClassifier()

# 그리드 서치를 통해 찾은 최적 하이퍼파라미터를 적용한다.
best_model.set_params(**xgboost_grid_clf.best_params_)

# 전체 학습 데이터를 사용하여 모델을 학습시킨다.
best_model.fit(train_X, train_y)

# 학습된 최종 모델을 출력하여 설정 상태를 확인한다.
best_model

In [ ]:
# 학습된 모델, 인코더, 스케일러를 지정된 경로에 pickle 형식으로 저장한다.
with open(best_model_path, 'wb') as fp:
    pickle.dump(best_model, fp)   # 최종 학습된 모델 저장
    pickle.dump(encoder1, fp)     # 인코더 객체 저장 (예: 범주형 변수 처리용)
    pickle.dump(scaler1, fp)      # 스케일러 객체 저장 (예: 정규화 또는 표준화용)

print('저장완료')  # 저장 완료 메시지 출력

In [ ]:
# 저장된 pickle 파일에서 모델, 인코더, 스케일러를 순서대로 불러온다.
with open(best_model_path, 'rb') as fp:
    test_model = pickle.load(fp)      # 저장된 최종 모델 불러오기
    test_encoder = pickle.load(fp)    # 저장된 인코더 불러오기
    test_scaler1 = pickle.load(fp)    # 저장된 스케일러 불러오기

In [ ]:
# 불러온 모델을 사용하여 학습 데이터에 대한 예측 결과를 생성한다.
r1 = test_model.predict(train_X)

# 예측 결과와 실제 레이블을 비교하여 F1 점수를 계산한다.
r2 = f1_score(train_y, r1)

# 계산된 F1 점수를 출력한다.
r2